<a href="https://colab.research.google.com/github/Kimdoyeon123/paper/blob/main/Inception_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

In [ ]:
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    conv3x3_reduce = Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)

    conv5x5_reduce = Conv2D(filters[3], (1,1), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[4], (5,5), padding = 'same', activation='relu')(conv5x5_reduce)

    pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv1x1, conv3x3, conv5x5, pool_proj], axis=-1)

    return output

In [ ]:
def googlenet(input_shape=(224, 224, 3), num_classes=1000):
    input_layer = Input(shape=input_shape)

    # Initial Conv and MaxPooling layers
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception Modules
    x = inception_module(x, [64, 96, 128, 16, 32, 32])  # Inception 3a
    x = inception_module(x, [128, 128, 192, 32, 96, 64])  # Inception 3b
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48, 64])  # Inception 4a
    x = inception_module(x, [160, 112, 224, 24, 64, 64])  # Inception 4b
    x = inception_module(x, [128, 128, 256, 24, 64, 64])  # Inception 4c
    x = inception_module(x, [112, 144, 288, 32, 64, 64])  # Inception 4d
    x = inception_module(x, [256, 160, 320, 32, 128, 128])  # Inception 4e
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128, 128])  # Inception 5a
    x = inception_module(x, [384, 192, 384, 48, 128, 128])  # Inception 5b

    # Final layers
    x = AveragePooling2D((7, 7), strides=(1, 1))(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(input_layer, x, name='googlenet')

    return model

# Create the model
model = googlenet(input_shape=(224, 224, 3), num_classes=1000)
model.summary()


Model: "googlenet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 56, 56, 64)           4160      ['max_pooling2d[0][0]'